# Capstone Project



**Team Members**

> Alexis Mercado 

> Gerardo Moreno

> Julio Sibria

# Download Data

download data ...

In [2]:
from google.colab import files

!pip install -q kaggle
# files.upload()
! mkdir ~/.kaggle

!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list


ref                                                           title                                              size  lastUpdated          downloadCount  
------------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                   Reddit Vaccine Myths                              233KB  2021-07-07 19:18:09           8954  
crowww/a-large-scale-fish-dataset                             A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           5482  
imsparsh/musicnet-dataset                                     MusicNet Dataset                                   22GB  2021-02-18 14:12:19           1974  
dhruvildave/wikibooks-dataset                                 Wikibooks Dataset                                   2GB  2021-07-03 18:37:20           2371  
promptcloud/careerbuilder-job-listing-2020                    Ca

In [3]:
# Download Data
!kaggle datasets download navoneel/brain-mri-images-for-brain-tumor-detection
!ls

 60% 9.00M/15.1M [00:00<00:00, 29.8MB/s]
100% 15.1M/15.1M [00:00<00:00, 38.2MB/s]
brain-mri-images-for-brain-tumor-detection.zip	kaggle.json  sample_data


In [4]:
!unzip brain-mri-images-for-brain-tumor-detection.zip

Archive:  brain-mri-images-for-brain-tumor-detection.zip
  inflating: brain_tumor_dataset/no/1 no.jpeg  
  inflating: brain_tumor_dataset/no/10 no.jpg  
  inflating: brain_tumor_dataset/no/11 no.jpg  
  inflating: brain_tumor_dataset/no/12 no.jpg  
  inflating: brain_tumor_dataset/no/13 no.jpg  
  inflating: brain_tumor_dataset/no/14 no.jpg  
  inflating: brain_tumor_dataset/no/15 no.jpg  
  inflating: brain_tumor_dataset/no/17 no.jpg  
  inflating: brain_tumor_dataset/no/18 no.jpg  
  inflating: brain_tumor_dataset/no/19 no.jpg  
  inflating: brain_tumor_dataset/no/2 no.jpeg  
  inflating: brain_tumor_dataset/no/20 no.jpg  
  inflating: brain_tumor_dataset/no/21 no.jpg  
  inflating: brain_tumor_dataset/no/22 no.jpg  
  inflating: brain_tumor_dataset/no/23 no.jpg  
  inflating: brain_tumor_dataset/no/24 no.jpg  
  inflating: brain_tumor_dataset/no/25 no.jpg  
  inflating: brain_tumor_dataset/no/26 no.jpg  
  inflating: brain_tumor_dataset/no/27 no.jpg  
  inflating: brain_tumor_datase

# Load the Data

Load data ...

In [18]:
# Load Data
import matplotlib.pyplot as plt
import glob
import cv2 as cv

yes_list = []
no_list = []

for filename in glob.glob('/content/yes/*.jpg'): 
    im = cv.imread(filename)
    yes_list.append(im)
    # plt.figure()
    # plt.imshow(im)

for filename in glob.glob('/content/no/*.jpg'): 
    im = cv.imread(filename)
    image_list.append(im)
    # plt.figure()
    # plt.imshow(im)

# EDA


Exploratory Data Analysis

In [ ]:
# EDA

# Train Test Split

Train Test Split ...

In [ ]:
# Train test split

# Model

Model ...

In [ ]:
# Build Model

# Making Predictions


Predictions ...

In [ ]:
#prediction